# Busca local e algoritmos genéticos

Neste notebook, realizaremos a implementação de algoritmos de algoritmos genéticos para problemas de busca local como coloração de mapas e problema das oito rainhas.


Vamos considerar o problema de coloração de grafos. Em particular, vamos considerar a instância Australia deste problema, conforme a figura abaixo.

![alt text](https://www.researchgate.net/profile/Abderrazak_Daoudi/publication/325808704/figure/fig2/AS:638325140512769@1529199981896/An-example-of-graph-coloring-problem.png)

Neste problema, o objetivo é colorir os territórios do mapa de tal forma que territórios vizinhos nunca tenham a mesma cor. No exemplo acima, é fácil ver que precisamos de pelo menos três cores para resolver o problema.

Para que o computador consiga resolver este problema, naturalmente precisamos representá-lo de uma forma mais eficiente do que uma imagem. Desta forma, podemos representar o problema como um grafo, onde os nós representam territórios e as arestas denotam territórios vizinhos.

De uma forma simples, o grafo acima pode ser representado através do dicionário a seguir, onde as chaves denotam o nome das arestas e os valores correspondentes denotam os nós conectados pela aresta. No caso abaixo, nós são representados por números, onde WA=0, NT=1, SA=2, Q=3, NSW=4, V=5 e T=6. Nota: nosso algoritmo assume que o grafo é [conectado](https://en.wikipedia.org/wiki/Connectivity_(graph_theory)#Connected_vertices_and_graphs), desta forma criamos uma aresta entre os nós V e T para o algoritmo funcionar.

In [ ]:
# Australia instance of the graph coloring problem; each entry represents an edge
australia = {
    'WA-NT': [0, 1],
    'WA-SA': [0, 2],
    'NT-SA': [1, 2],
    'NT-Q': [1, 3],
    'SA-Q': [2, 3],
    'SA-NSW': [2, 4],
    'SA-V': [2, 5],
    'Q-NSW': [3, 4],
    'NSW-V': [4, 5],
    'V-T': [5, 6]
}



## Algoritmos genéticos

Abaixo, temos a implementação de um algoritmo genético. O algoritmo recebe uma população inicial (`population`), uma função de avaliação de soluções (`fn_fitness`), um conjunto de valores possíveis para cada posição da solução (`gene_pool`), uma função de critério de parada por qualidade da solução (`fn_thres`), um número máximo de gerações (`ngen`) e uma probabilidade de mutação (`pmut`). Ao final, o algoritmo retorna o melhor indivíduo (solução) da última geração de indivíduos.

In [ ]:
import random as random
import bisect

def genetic_algorithm(population, fn_fitness, gene_pool, fn_thres=None, ngen=1000, pmut=0.1):

    # for each generation
    for i in range(ngen):

        # create a new population
        new_population = []

        # repeat to create len(population) individuals
        for i in range(len(population)):

          # select the parents
          p1, p2 = select(2, population, fn_fitness)

          # recombine the parents, thus producing the child
          child = recombine(p1, p2)

          # mutate the child
          child = mutate(child, gene_pool, pmut)

          # add the child to the new population
          new_population.append(child)

        # move to the new population
        population = new_population

        # check if one of the individuals achieved a fitness of fn_thres; if so, return it
        fittest_individual = fitness_threshold(fn_fitness, fn_thres, population)
        if fittest_individual:
            return fittest_individual

    # return the individual with highest fitness
    return max(population, key=fn_fitness)

# get the best individual of the received population and return it if its
# fitness is higher than the specified threshold fn_thres
def fitness_threshold(fn_fitness, fn_thres, population):
    if not fn_thres:
        return None

    fittest_individual = max(population, key=fn_fitness)
    if fn_fitness(fittest_individual) >= fn_thres:
        return fittest_individual

    return None

Para o algoritmo acima funcionar, precisamos das seguintes funções de apoio: `fn_fitness`, `select`, `recombine` e `mutate`. A primeira função deve ser definida com base na definição do problema que queremos resolver (como no exemplo anterior, onde foi criada a função `fn_evaluate_GC` para o problema de coloração de grafos). As demais funções, por outro lado, tendem a ser mais genéricas, representando o que chamamos de operadores genéticos. Naturalmente, nem todos operadores genéticos funcionam bem em todos os problemas. Desta forma, devemos escolhar os operadores genéticos com base no problema em questão.

A função `select` abaixo implementa o operador genético de seleção. Em particular, é implementada a seleção da roleta (ou seleção proporcional à aptidão). Neste método, a partir da população (`population`) e função de aptidão (`fn_fitness`) recebidas, o método retorna `r` indivíduos aleatoriamente, onde a probabilidade de um dado indivíduo ser selecionado é proporcional à sua aptidão. Este tipo de seleção faz bastante sentido na prática, pois garante que indivíduos mais aptos tenham chances maiores de produzir filhos.

In [ ]:
# genetic operator for selection of individuals;
# this function implements roulette wheel selection, where individuals with
# higher fitness are selected with higher probability
def select(r, population, fn_fitness):
    fitnesses = map(fn_fitness, population)
    sampler = weighted_sampler(population, fitnesses)
    return [sampler() for i in range(r)]

# return a function that whenever called returns a single sample from seq;
# the probability of a sample being returned is proportional to its weight
def weighted_sampler(seq, weights):
    totals = []
    for w in weights:
        totals.append(w + totals[-1] if totals else w)
    return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]

A função `recombine` abaixo implementa o operador genético de recombinação (ou cruzamento). Em particular, a função implementa a recombinação de ponto único, onde um número aleatório `c` é gerado para dividir as porções iniciais (índices 0 a c) e finais (índices c a n) dos pais (`x` e `y`). O filho é então produzido a partir da porção inicial de um dos pais e a porção final do outro.

In [ ]:
# genetic operator for recombination (crossover) of individuals;
# this function implements single-point crossover, where the resulting individual
# carries a portion [0,c] from parent x and a portion [c,n] from parent y, with
# c selected at random
def recombine(x, y):
    n = len(x)
    c = random.randrange(0, n)
    return x[:c] + y[c:]

A função `mutate` abaixo implementa o operador genético de mutação. Em particular, a função implementa a mutação uniforme no indivíduo recebido `x`. Nesta mutação, um gene (posição) do indivíduo é selecionado de forma aleatória e seu valor é substituído por um dos valores possíveis (`gene_pool`) aleatoriamente. Vale observar que o indivíduo `x` só sofre mutação com probabilidade `pmut`. Note ainda que neste método apenas um gene sofre mutação.

In [ ]:
# genetic operator for mutation;
# this function implements uniform mutation, where a single element of the
# individual is selected at random and its value is changed by a randomly chosen
# value (out of the possible values in gene_pool)
def mutate(x, gene_pool, pmut):

    # if random >= pmut, then no mutation is performed
    if random.uniform(0, 1) >= pmut:
        return x

    c = random.randrange(0, len(x)) # gene to be mutated
    r = random.randrange(0, len(gene_pool)) # new value of the selected gene
    new_gene = gene_pool[r]
    return x[:c] + [new_gene] + x[c+1:]

Definidos os operadores acima, agora precisamos criar um método para gerar a população inicial. Esta população deve ser gerada de forma aleatória. A função `init_population` faz exatamente isso. Para tal, ela recebe três parâmetros. O primeiro (`pop_number`) especifica o tamanho da população. O segundo (`gene_pool`) define os valores possíveis para os genes de cada indivíduo. O terceiro (`state_length`) denota o número de genes de cada indivíduo. Ao final, a função abaixo retorna uma lista com a população de indivíduos.

In [ ]:
def init_population(pop_number, gene_pool, state_length):
    g = len(gene_pool)
    population = []

    for i in range(pop_number):
        # each individual is represented as an array with size state_length,
        # where each position contains a value from gene_pool selected at random
        new_individual = [gene_pool[random.randrange(0, g)] for j in range(state_length)]
        population.append(new_individual)

    return population

Agora podemos nos preocupar com o problema em si. Vamos considerar novamente o problema de coloração de grafos visto anteriormente.

In [ ]:
# Australia instance of the graph coloring problem; each entry represents an edge
problem_instance = {
    'WA-NT': [0, 1],
    'WA-SA': [0, 2],
    'NT-SA': [1, 2],
    'NT-Q': [1, 3],
    'SA-Q': [2, 3],
    'SA-NSW': [2, 4],
    'SA-V': [2, 5],
    'Q-NSW': [3, 4],
    'NSW-V': [4, 5],
    'V-T': [5, 6]
}

# set of possible colours (red, green, blue)
possible_values = ['R', 'G', 'B']

# length of an individual (one position per territory)
individual_length = 7

Para avaliar a aptidão dos indivíduos no problema de coloração de grafos, podemos utilizar a mesma classe que vimos anteriomente. Esta classe calcula a aptdidão de um indivíduo como o número de pares de territórios vizinhos que possuem cores diferentes. Lembrando que quanto maior este número, melhor.

In [ ]:
# evaluation class;
# since that a solution needs to be evaluated with respect to the problem instance
# in consideration, we created this class to store the problem instance and to
# allow the evaluation to be performed without having the problem instance at hand
class EvaluateGC:
    # during initialization, store the problem instance
    def __init__(self, problem_instance):
        self.problem_instance = problem_instance

    # compute the value of the received solution
    def __call__(self, solution):
        # counts the number of edges that connect vertices of the same colour
        return sum(solution[n1] != solution[n2] for (n1, n2) in self.problem_instance.values())

# create an instance of the evaluation class for the considered problem instance
fn_fitness = EvaluateGC(australia)
# now, a solution X can be evaluated by simply calling fn_fitness(X)

Agora, finalmente, podemos testar nosso algoritmo.

In [ ]:
# population size
population_size = 8

# initial population
population = init_population(population_size, possible_values, individual_length)

# run the algoritm
solution = genetic_algorithm(population, fn_fitness, gene_pool=possible_values, fn_thres=10)

# print the results
print('Resulting solution: %s' % solution)
print('Value of resulting solution: %d' % fn_fitness(solution))

Resulting solution: ['G', 'R', 'B', 'G', 'R', 'G', 'R']
Value of resulting solution: 10


Se você testar o código acima, verá que o algoritmo genético consegue encontrar o resultado ótimo com uma certa facilidade nesta instância do problema de coloração de grafos. Lembrando que, nesta instância, a solução ótima possui um valor de 10.

Novamente, lembre que o problema que estamos resolvendo aqui é o de coloração de grafos. Da mesma forma que vimos anteriormente, podemos utilizar este algoritmo para resolver outras instâncias do mesmo problema, bastando especificar as variáveis `problem_instance`, `possible_values`, `individual_length` e `fn_fitness` correspondentes.

Naturalmente, dependendo do problema a ser considerado, pode ser necessário ajustar também alguns elementos do algoritmo, como os operadores genéticos. Os operadores genéticos sempre devem ser escolhidos com base na estrutura do problema.

### Exercício 3

Utilize o algoritmo genético para resolver a instância "grafo de Petersen" do problema de coloração de grafos da última aula. O número de cores possíveis aqui continua sendo 3 e o valor ótimo permanece sendo 15.

In [ ]:
# Petersen graph
petersen = {
    # TODO: your answer here (15 lines)
}

# set of possible colours (red, green, blue)
possible_values = ['R', 'G', 'B']

# length of an individual (one position per territory)
individual_length = # TODO: your answer here

# create an instance of the evaluation class for the considered problem instance
fn_fitness = EvaluateGC(petersen)

# population size
population_size = 8

# initial population
population = init_population(population_size, possible_values, individual_length)

# run the algoritm
solution = genetic_algorithm(population, fn_fitness, gene_pool=possible_values, fn_thres=10)

# print the results
print('Resulting solution: %s' % solution)
print('Value of resulting solution: %d' % fn_fitness(solution))

### Exercício 4

Utilize o algoritmo genético para resolver a instância 8-rainhas do problema n-rainhas. Lembrando que na solução ótima nenhuma rainha consegue atacar as demais.

Primeiramente, implemente a classe `EvaluateNQueens`. Note que nossa implementação do operador de seleção não lida bem com valores negativos. Desta forma, devemos implementar o método `__call__` de modo a retornar um valor no intervalo $[0,max\_conflicts]$. O número máximo de conflitos no n-Queens é $\frac{n(n-1)}{2}$. Exemplo: para $n=8$, temos no máximo 28 conflitos; logo, ao receber uma solução com 7 conflitos, a função deve retornar 21.

In [ ]:
# evaluation class for the n-Queens problem
class EvaluateNQueens:
    # during initialization, store the problem instance
    def __init__(self, n):
        self.n = n

    # compute the value of the received solution
    def __call__(self, solution):
        conflicts = 0

        # TODO: count the number of conflicts (7 lines) and calculate fitness

        # our genetic algorithm implementation maximises the value;
        # moreover, its selection operator does not handle negative values properly;
        # thus, we estimate the maximum possible number of conflicts (which is
        # given by n(n-1)/2) and subtract from it the number of conflicts of the
        # received solution; this allows our algorithm to maximize the fitness;
        # hence, our objective here is to get as close to n(n-1)/2 as possible
        return fitness

Agora, implemente a instanciação do problema, configure o algoritmo genético e execute o mesmo.

In [ ]:
# instance of the N-Queens problem
n = 8

# create an instance of the evaluation class for the considered problem instance
fn_fitness = EvaluateNQueens(n)

# set of possible values
possible_values = # TODO: your answer here

# initial state (we begin with a solution where all queens are on the same row)
init_state = # TODO: your answer here

# population size
population_size = 8

# initial population
population = init_population(population_size, possible_values, n)

# run the algoritm
solution = genetic_algorithm(population, fn_fitness, gene_pool=possible_values, fn_thres=28, ngen=1000)

# print the results
print('Resulting solution: %s' % solution)
print('Value of resulting solution (the higher the better): %d' % fn_fitness(solution))

### Exercício 5

Implemente o operador de mutação do algoritmo genético. Existem diversas possibilidades aqui. Uma sugestão é a mutação baseada em troca de posições (swap), onde duas posições são selecionadas aleatoriamente e seus valores são trocados. Mais detalhes (e outras opções) em: [link 1](https://link.springer.com/chapter/10.1007/978-1-4614-6940-7_4), [link 2](https://www.inf.ufsc.br/~alexandre.goncalves.silva/courses/14s2/ine5633/slides/aulaAG.pdf), [link 3](https://sites.icmc.usp.br/andre/research/genetic/) e [link 4](http://www2.peq.coppe.ufrj.br/Pessoal/Professores/Arge/COQ897/Naturais/aulas_piloto/aula4.pdf). Em seguida, rode o algoritmo nos problemas dos dois exercícios anteriores e compare o resultado das execuções do algoritmo genético com a mutação original e a criada aqui.

In [ ]:
def mutate(x, gene_pool, pmut):
    # TODO: your answer here
    return x

Agora, execute o algoritmo genético anterior com o novo operador de mutação.

In [ ]:
# run the algoritm
solution = genetic_algorithm(population, fn_fitness, gene_pool=possible_values, fn_thres=28, ngen=1000)

# print the results
print('Resulting solution: %s' % solution)
print('Value of resulting solution (the higher the better): %d' % fn_fitness(solution))